In [1]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np

from model_new import BertForTokenClassification
import utils.NERutils as nu

from transformers import AutoConfig, AutoTokenizer

#### Define tokenizer

In [2]:
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

#### Load Datasets

In [3]:
train_path = "data/train.parquet"
dev_path = "data/dev.parquet"
test_path = "data/test.parquet"

In [4]:
train_dataset = nu.NERdataset(dataset_path=train_path, tokenizer=bert_tokenizer)
dev_dataset = nu.NERdataset(dataset_path=dev_path, tokenizer=bert_tokenizer)
test_dataset = nu.NERdataset(dataset_path=test_path, tokenizer=bert_tokenizer)

#### Get pretrained model

In [5]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [6]:
# Config
bert_model_name = "bert-base-multilingual-cased"
bert_config = AutoConfig.from_pretrained(
    bert_model_name, 
    num_labels=len(train_dataset.tags), 
    id2label=train_dataset.index2tag, 
    label2id=train_dataset.tag2index
)

model = BertForTokenClassification.from_pretrained(bert_model_name, config=bert_config, tags=train_dataset.tags).to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Finetune

In [7]:
train_params = {'batch_size': 8,
                'shuffle': True,
                'num_workers': 0,
                }

data_loader = torch.utils.data.DataLoader(train_dataset, **train_params)

In [9]:
num_epochs = 1
learning_rate = 1e-05
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

model.finetune(num_epochs, data_loader, device, optimizer)

  0%|          | 0/1471 [00:00<?, ?it/s]

: 

In [ ]:
model.training_loss